In [179]:
import os
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
#!pip install seaborn
import seaborn as sns

#!pip install imblearn
#if the above command does not work to install imblearn package run the following command in your terminal
# conda install -c glemaitre imbalanced-learn
from imblearn.over_sampling import SMOTE#for class imbalance.
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.metrics import mean_squared_error
from math import sqrt


import warnings
warnings.filterwarnings('ignore')

In [163]:
train_CSV1="D:/INSOFY/MITH/Train and Test Data-1572653937434/Data_For_all/train1.csv"
train_CSV2="D:/INSOFY/MITH/Train and Test Data-1572653937434/Data_For_all/train2.csv"
train_CSV3="D:/INSOFY/MITH/Train and Test Data-1572653937434/Data_For_all/train3.csv"
train_Distance_CSV="D:/INSOFY/MITH/Train and Test Data-1572653937434/Data_For_all/building_to_mrt_distance.csv"

test_CSV ="D:/INSOFY/MITH/Train and Test Data-1572653937434/Data_For_all/test.csv"

In [107]:
#Function Read csv file.
# _csvTrainData <- csv file directory.
def ReadData(_csvTrainData):
    passed_DataFrame=pd.read_csv(_csvTrainData, sep=',', header=0, na_values='unknown')
    return passed_DataFrame

# Function: Describe data characteristics.
# _df <- data frame.
# _catData <- boolean value to know the passed data is cat variable
def DescribeData(_df,_catData,_name="DataFrame"):
    print("\n",_name,"Shape")
    print(_df.shape)
    print("\n",_name,"HEAD")
    print(display(_df.head()))
    print("\n",_name,"D-TYPES")
    print(_df.dtypes)
    print("\n",_name,"DESCRIBE")
    print(display(_df.describe()))
    if _catData==True:
        print(display(_df.describe(include=['object'])))
        
# Function: Find missing Data in the data frame.
# _isPercentage <- boolian value to be passed to know the % of missing values in all attributes.
# _name <- name(string) name of the data frame.
def FindMissingData(_df,_isPercentage,_name="DataFrame"):
    if _isPercentage==False:
        print(_df.isnull().sum())
    else:
        percent_missing = round((_df.isnull().sum()/ len(_df))*100,3)
        missing_value_df = pd.DataFrame({'column_name': _df.columns,'percent_missing': percent_missing})
        missing_value_df.set_index('column_name', inplace=True)
        print(display(missing_value_df))
    
# 
# Function: Get value counts.
# _df <- data frame
def GetValueCounts(_df):
    print(_df.value_counts())
    #_df[_df.y == 'yes'].marital.value_counts(normalize=True) //To get the normalized values.

# Function: Change data types.
# _df <- data frame
# _attributes <- attributes that are subjected to change are passed as a list.
# _toType <- convertion data type. ------ int64,float64,category.
def ChangeDataType(_df,_attributes,_toType):
    for col in _attributes:
        _df[col] = _df[col].astype(_toType)
    print(_df.dtypes)
    
# Function: Drop attributes. 
# _df <- data frame
# _dropAttributes <- Attribute that is to be dropped.
def DropAttributes(_df,_dropAttributes):
    _df.drop(_dropAttributes,axis = 1, inplace= True)
    print(_df.columns)

# Function to replace values in data set.
# _df <- data frame
# _fromOption <- value subjected to change.
# _toOption <- value change
def ReplaceOptions(_df,_fromOption,_toOption):
    _df.replace(_fromOption,_toOption,inplace=True)
    
# Function to Group data based on an attribute.
# _isAggrigate is for aggrigating the numeric values based on the attribute to group.
# _df <- data frame
# _groupId <- attribute to group the dataframe.
def GetGroupedData(_df,_groupId,_isAggrigate=False):
    if _isAggrigate:
        _df = _df.groupby(_groupId).sum().reset_index()
    else:
        _df = _df.groupby(_groupId).reset_index()
        
    return _df

# Function for merging different data frame.
# _df <- dataframe
def MergeData(_df1,_df2,_mergeAttribute):
    _merge_df = pd.merge(_df1,_df2,on=_mergeAttribute)
    return _merge_df

# Function to export final predictions as CSV.
# _indexID <- Index Attribute(Column)
# __predAttribute <- Predicated attribute (Column)
# _label1 <- name of column1
# _label2 <- name of column2
def ExportFinal_CSV(_indexID,_predAttribute,_label1,_label2,_FileName='Mith_Predication.csv'):
    Mith_Predication = pd.DataFrame(columns=[_label1,_label2])
    Mith_Predication[_label1] = _indexID
    Mith_Predication[_label2] = _predAttribute
    Mith_Predication.to_csv(_FileName,index=True)
    return Mith_Predication

# Function to split date attribute and return a data frame.
# _dateAttribute <- date column
def GetDate_df(_dateAttribute):
    Date_df = pd.DataFrame(columns=["Date"])
    Date_df['year'] = _dateAttribute.dt.year
    Date_df['month'] = _dateAttribute.dt.month 
    Date_df['day'] = _dateAttribute.dt.dayofyear 
    Date_df['weekday'] = _dateAttribute.dt.weekday
    return Date_df

def GetyearFromDf(_df,_attribute):
    _df[_attribute]=_df[_attribute].astype("category")
    data = [x[0] for x in _df[_attribute].str.split('-')]
    _df[_attribute]=data
    _df[_attribute]=_df[_attribute].astype("int64")   

In [108]:
def DisplayQuantityPlot(_df,_data):
    sns.set(rc={'figure.figsize':(19,8.27)})
    sns.countplot(x=_data,data=_df)
    plt.show()
    
def GenerateHeatMap(_df):
    corr = _df.corr()
    ax = sns.heatmap(corr,annot =True, linewidth = 5, cbar = True)
    
def GenerateDistPlot(_df,_data):
    sns.distplot(_df[_data])
    
def generateBoxPlot(_df,_sortbyData,_measureData):
    _df.boxplot(by =_sortbyData, column =[_measureData], grid = False, figsize=(15, 10)) 
    
def DrawBarChart(_df,_indVar,_depVar,_yColName,_xColName,_title):
    plt.bar(_indVar,_depVar) 
    plt.xlabel(_xColName) 
    plt.ylabel(_yColName) 
    plt.title(_title)
    plt.show() 

def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [164]:
train_df1=ReadData(train_CSV1)
train_df2=ReadData(train_CSV2)
train_df3=ReadData(train_CSV3)
train_Distance_df=ReadData(train_Distance_CSV)

test_df = ReadData(test_CSV)

In [177]:
test_df1 = ReadData(test_CSV)

In [110]:
DescribeData(train_df1,False,"train_df1")


 train_df1 Shape
(52203, 11)

 train_df1 HEAD


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,uniqueID
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,100000
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,100001
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,100002
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,100003
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,100004


None

 train_df1 D-TYPES
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
uniqueID                 int64
dtype: object

 train_df1 DESCRIBE


,floor_area_sqm,lease_commence_date,resale_price,uniqueID
count,52203.000000,52203.000000,5.220300e+04,52203.000000
mean,96.203285,1989.512902,4.612147e+05,126101.000000
std,24.984010,10.139550,1.225534e+05,15069.852388
min,31.000000,1966.000000,1.950000e+05,100000.000000
25%,73.000000,1983.000000,3.700000e+05,113050.500000
50%,95.000000,1988.000000,4.400000e+05,126101.000000
75%,111.000000,1999.000000,5.250000e+05,139151.500000
max,266.000000,2012.000000,1.088888e+06,152202.000000


None


In [111]:
DescribeData(train_df2,False,"train_df2")


 train_df2 Shape
(37153, 12)

 train_df2 HEAD


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,uniqueID
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,152203
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,152204
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,152205
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,152206
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,152207


None

 train_df2 D-TYPES
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease          int64
resale_price           float64
uniqueID                 int64
dtype: object

 train_df2 DESCRIBE


,floor_area_sqm,lease_commence_date,remaining_lease,resale_price,uniqueID
count,37153.000000,37153.000000,37153.000000,3.715300e+04,37153.000000
mean,97.020386,1990.920195,73.913116,4.368628e+05,170779.000000
std,24.198360,10.862330,10.885456,1.358052e+05,10725.291612
min,31.000000,1966.000000,48.000000,1.900000e+05,152203.000000
25%,74.000000,1984.000000,66.000000,3.400000e+05,161491.000000
50%,96.000000,1989.000000,72.000000,4.080000e+05,170779.000000
75%,111.000000,2000.000000,83.000000,4.950000e+05,180067.000000
max,280.000000,2013.000000,97.000000,1.150000e+06,189355.000000


None


In [112]:
DescribeData(train_df3,False,"train_df3")


 train_df3 Shape
(32578, 12)

 train_df3 HEAD


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,uniqueID
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,189356
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,189357
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,189358
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,189359
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,189360


None

 train_df3 D-TYPES
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
uniqueID                 int64
dtype: object

 train_df3 DESCRIBE


,floor_area_sqm,lease_commence_date,resale_price,uniqueID
count,32578.000000,32578.000000,3.257800e+04,32578.000000
mean,98.247861,1993.053011,4.450976e+05,205644.500000
std,24.227319,11.953732,1.530538e+05,9404.602871
min,31.000000,1966.000000,1.700000e+05,189356.000000
25%,82.000000,1984.000000,3.350000e+05,197500.250000
50%,97.000000,1993.000000,4.100000e+05,205644.500000
75%,113.000000,2002.000000,5.180000e+05,213788.750000
max,249.000000,2016.000000,1.180000e+06,221933.000000


None


In [113]:
DescribeData(train_Distance_df,False,"train_Distance_df")


 train_Distance_df Shape
(8811, 120)

 train_Distance_df HEAD


,Building,Jurong East MRT station,Bukit Batok MRT station,Bukit Gombak MRT station,Choa Chu Kang MRT station,Yew Tee MRT station,Kranji MRT station,Marsiling MRT station,Woodlands MRT station,Admiralty MRT station,...,Bendemeer MRT station,Geylang Bahru MRT Station,Mattar MRT station,Ubi MRT station,Kaki Bukit MRT Station,Bedok North MRT station,Bedok Reservoir MRT station,Tampines West MRT station,Tampines East MRT station,Upper Changi MRT station
0,ANCHORVALE ST BLK 331C,17818.84265,16471.03384,15856.79247,16199.22480,15808.72792,14570.109930,13485.500120,12369.979820,11061.899320,...,9584.39327,8494.000192,7640.691202,7398.618897,7079.585070,7440.940807,8107.447171,7743.354078,8438.549604,9950.182657
1,PUNGGOL WAY BLK 266B,19092.61457,17694.05674,17032.15623,17205.95322,16731.88625,15248.533120,14065.860790,12863.730930,11445.713010,...,10932.92815,9808.425468,8872.302279,8425.747280,7957.972942,8163.873942,8589.377565,8042.543847,8366.769609,9996.575353
2,YISHUN AVE 9 BLK 318A,15919.39969,14166.65357,13182.50154,12293.68148,11439.72263,9122.636441,7722.266883,6368.157244,4795.908711,...,13570.68606,12907.865810,12661.750790,13179.447730,13208.994580,13791.613210,14675.215580,14401.138090,15072.755350,16621.097390
3,YISHUN AVE 9 BLK 318B,15903.04510,14152.90288,13171.66285,12296.13129,11447.44104,9144.786709,7748.983596,6398.540744,4831.224121,...,13515.78029,12851.563270,12604.181940,13121.274190,13151.040900,13734.103470,14618.776730,14345.977870,15020.740460,16567.813760
4,YISHUN AVE 9 BLK 315A,15705.88647,13955.91314,12975.36314,12109.29884,11267.18726,8991.283610,7607.087576,6267.390055,4716.850722,...,13402.90599,12752.522990,12524.591610,13068.535130,13115.059640,13710.617970,14613.647670,14356.366340,15058.662370,16595.614410


None

 train_Distance_df D-TYPES
Building                          object
Jurong East MRT station          float64
Bukit Batok MRT station          float64
Bukit Gombak MRT station         float64
Choa Chu Kang MRT station        float64
Yew Tee MRT station              float64
Kranji MRT station               float64
Marsiling MRT station            float64
Woodlands MRT station            float64
Admiralty MRT station            float64
Sembawang MRT station            float64
Yishun MRT station               float64
Khatib MRT station               float64
Yio Chu Kang MRT station         float64
Ang Mo Kio MRT station           float64
Bishan MRT station               float64
Braddell MRT station             float64
Toa Payoh MRT station            float64
Novena MRT station               float64
Newton MRT station               float64
Orchard MRT station              float64
Somerset MRT station             float64
Dhoby Ghaut MRT station          float64
City Hall MRT station   

,Jurong East MRT station,Bukit Batok MRT station,Bukit Gombak MRT station,Choa Chu Kang MRT station,Yew Tee MRT station,Kranji MRT station,Marsiling MRT station,Woodlands MRT station,Admiralty MRT station,Sembawang MRT station,...,Bendemeer MRT station,Geylang Bahru MRT Station,Mattar MRT station,Ubi MRT station,Kaki Bukit MRT Station,Bedok North MRT station,Bedok Reservoir MRT station,Tampines West MRT station,Tampines East MRT station,Upper Changi MRT station
count,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,...,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000
mean,13342.501418,12442.126596,12142.940232,12852.327528,12953.079194,13233.871906,12934.203488,12597.673452,12289.558442,12485.753955,...,10411.821946,10113.638971,10120.111528,10652.021687,10889.945022,11410.583848,12296.895165,12459.980132,13680.274099,14576.258618
std,6584.103046,6377.638618,6268.942906,6573.108811,6401.803397,5901.272718,5712.753199,5539.729359,5322.884872,5118.800316,...,4578.419474,4853.460059,5294.375534,6030.719639,6436.789482,6836.128044,7399.167740,7647.228450,8125.701227,8140.029411
min,387.601779,89.211152,80.830430,176.087487,82.171822,1184.236035,58.788000,328.395798,89.766456,79.747823,...,234.600752,29.550377,301.794871,111.392775,41.255827,95.785596,148.695119,56.703463,58.065331,256.468034
25%,7094.385092,6359.631530,6937.728977,7436.025738,7710.325131,8668.768253,8338.395631,8824.131432,9295.872289,9958.870180,...,6806.606729,6276.738680,5834.168787,5703.106364,5600.851986,5656.786675,6337.638813,6177.309939,7202.543135,8428.900645
50%,13802.548050,12480.900690,12093.586740,12771.510790,13120.142150,13669.493920,13509.192440,13329.384650,12834.894290,12775.611680,...,10708.771250,9699.713616,8777.938785,8186.306088,7934.144002,8852.886215,10276.114880,10690.940000,12492.130450,13262.254670
75%,18308.971470,17263.308160,16907.955220,17523.388145,17317.925730,17270.449530,17108.359350,16830.686325,16563.624190,16888.336505,...,13905.756705,14430.337705,15229.677290,16422.698435,16929.149950,17810.230110,19155.702110,19397.867365,20924.076995,21904.792140
max,28006.826820,26899.192660,26443.092250,27074.284180,26738.340580,25435.619390,24256.825680,23035.604000,21570.601300,20148.676200,...,20001.282230,20820.247310,22030.332660,23916.802420,24892.133250,25934.100480,27517.143390,28136.322910,30008.651220,30719.786960


None


In [165]:
DescribeData(test_df,False,"test_df")


 test_df Shape
(21846, 11)

 test_df HEAD


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,uniqueID
0,2018-08,ANG MO KIO,2 ROOM,323,ANG MO KIO AVE 3,01 TO 03,44.0,Improved,1977,57 years 10 months,221934
1,2018-08,ANG MO KIO,3 ROOM,220,ANG MO KIO AVE 1,07 TO 09,82.0,New Generation,1977,57 years 11 months,221935
2,2018-08,ANG MO KIO,3 ROOM,220,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1977,57 years 11 months,221936
3,2018-08,ANG MO KIO,3 ROOM,445,ANG MO KIO AVE 10,01 TO 03,67.0,New Generation,1979,60 years 01 month,221937
4,2018-08,ANG MO KIO,3 ROOM,471,ANG MO KIO AVE 10,01 TO 03,67.0,New Generation,1979,59 years 11 months,221938


None

 test_df D-TYPES
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
uniqueID                 int64
dtype: object

 test_df DESCRIBE


,floor_area_sqm,lease_commence_date,uniqueID
count,21846.000000,21846.000000,21846.000000
mean,97.425744,1993.758354,232856.500000
std,24.250775,12.643644,6306.541326
min,31.000000,1966.000000,221934.000000
25%,81.000000,1984.000000,227395.250000
50%,95.000000,1993.000000,232856.500000
75%,112.000000,2003.000000,238317.750000
max,237.000000,2016.000000,243779.000000


None


#### Train 1 data set dosent have remainig lease as of the other data set, so trying to get the remaing lease values with month & Lease commerse date

In [114]:
train_df1.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,uniqueID
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,100000
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,100001
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,100002
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,100003
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,100004


In [115]:
GetyearFromDf(train_df1,"month")

In [116]:
train_df1.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,uniqueID
0,2012,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,100000
1,2012,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,100001
2,2012,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,100002
3,2012,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,100003
4,2012,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,100004


In [117]:
train_df1.dtypes

month                    int64
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
uniqueID                 int64
dtype: object

In [118]:
train_df1['remaining_lease']=np.abs((train_df1["month"]-train_df1["lease_commence_date"])-99)

In [119]:
train_df1.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,uniqueID,remaining_lease
0,2012,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,100000,73
1,2012,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,100001,67
2,2012,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,100002,67
3,2012,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,100003,71
4,2012,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,100004,67


#### Converting the data into their respective data types.

In [120]:
train1_attributesName_Category=['town','block','flat_type', 'street_name', 'storey_range', 'flat_model','uniqueID']
train1_attributesName_Numeric=['month','lease_commence_date','resale_price','remaining_lease','floor_area_sqm']
ChangeDataType(train_df1,train1_attributesName_Category,'category')
ChangeDataType(train_df1,train1_attributesName_Numeric,'int64')

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm          float64
flat_model             category
lease_commence_date       int64
resale_price            float64
uniqueID               category
remaining_lease           int64
dtype: object
month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
resale_price              int64
uniqueID               category
remaining_lease           int64
dtype: object


In [121]:
train_df1["block"]=train_df1["block"].astype("category")
train_df1["storey_range"]=train_df1["storey_range"].astype("category")
train_df1["uniqueID"]=train_df1["uniqueID"].astype("category")

In [122]:
train_df1.dtypes

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
resale_price              int64
uniqueID               category
remaining_lease           int64
dtype: object

In [123]:
FindMissingData(train_df1,True,"train_df1")

,percent_missing
column_name,
month,0.0
town,0.0
flat_type,0.0
block,0.0
street_name,0.0
storey_range,0.0
floor_area_sqm,0.0
flat_model,0.0
lease_commence_date,0.0


None


In [124]:
train_df1.shape

(52203, 12)

#### Analysing train data 2

In [125]:
train_df2.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease          int64
resale_price           float64
uniqueID                 int64
dtype: object

In [126]:
train_df2.shape

(37153, 12)

In [127]:
GetyearFromDf(train_df2,"month")

In [128]:
train_df2.dtypes

month                    int64
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease          int64
resale_price           float64
uniqueID                 int64
dtype: object

In [129]:
train_df2.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,uniqueID
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,152203
1,2015,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,152204
2,2015,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,152205
3,2015,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,152206
4,2015,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,152207


In [130]:
ChangeDataType(train_df2,train1_attributesName_Category,'category')
ChangeDataType(train_df2,train1_attributesName_Numeric,'int64')
train_df2["block"]=train_df2["block"].astype("category")
train_df2["storey_range"]=train_df2["storey_range"].astype("category")
train_df2["uniqueID"]=train_df2["uniqueID"].astype("category")

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm          float64
flat_model             category
lease_commence_date       int64
remaining_lease           int64
resale_price            float64
uniqueID               category
dtype: object
month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
remaining_lease           int64
resale_price              int64
uniqueID               category
dtype: object


In [131]:
train_df2.dtypes

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
remaining_lease           int64
resale_price              int64
uniqueID               category
dtype: object

In [132]:
FindMissingData(train_df2,True,"train_df2")

,percent_missing
column_name,
month,0.0
town,0.0
flat_type,0.0
block,0.0
street_name,0.0
storey_range,0.0
floor_area_sqm,0.0
flat_model,0.0
lease_commence_date,0.0


None


#### Analysing Train data 3

In [133]:
GetyearFromDf(train_df3,"month")

In [134]:
train_df3.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,uniqueID
0,2017,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,189356
1,2017,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,189357
2,2017,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,189358
3,2017,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,189359
4,2017,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,189360


##### removing months from Remaining lease attribute

In [135]:
train_df3['remaining_lease'] = train_df3['remaining_lease'].str.split(' ').str[0]

In [136]:
train_df3.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,uniqueID
0,2017,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61,232000.0,189356
1,2017,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60,250000.0,189357
2,2017,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,262000.0,189358
3,2017,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62,265000.0,189359
4,2017,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,265000.0,189360


In [137]:
ChangeDataType(train_df3,train1_attributesName_Category,'category')
ChangeDataType(train_df3,train1_attributesName_Numeric,'int64')
train_df3["block"]=train_df3["block"].astype("category")
train_df3["storey_range"]=train_df3["storey_range"].astype("category")
train_df3["uniqueID"]=train_df3["uniqueID"].astype("category")

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm          float64
flat_model             category
lease_commence_date       int64
remaining_lease          object
resale_price            float64
uniqueID               category
dtype: object
month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
remaining_lease           int64
resale_price              int64
uniqueID               category
dtype: object


In [138]:
train_df3.dtypes

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
remaining_lease           int64
resale_price              int64
uniqueID               category
dtype: object

#### Analysing Test data

In [166]:
GetyearFromDf(test_df,"month")

In [167]:
test_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,uniqueID
0,2018,ANG MO KIO,2 ROOM,323,ANG MO KIO AVE 3,01 TO 03,44.0,Improved,1977,57 years 10 months,221934
1,2018,ANG MO KIO,3 ROOM,220,ANG MO KIO AVE 1,07 TO 09,82.0,New Generation,1977,57 years 11 months,221935
2,2018,ANG MO KIO,3 ROOM,220,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1977,57 years 11 months,221936
3,2018,ANG MO KIO,3 ROOM,445,ANG MO KIO AVE 10,01 TO 03,67.0,New Generation,1979,60 years 01 month,221937
4,2018,ANG MO KIO,3 ROOM,471,ANG MO KIO AVE 10,01 TO 03,67.0,New Generation,1979,59 years 11 months,221938


In [168]:
test_df['remaining_lease'] = test_df['remaining_lease'].str.split(' ').str[0]
test_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,uniqueID
0,2018,ANG MO KIO,2 ROOM,323,ANG MO KIO AVE 3,01 TO 03,44.0,Improved,1977,57,221934
1,2018,ANG MO KIO,3 ROOM,220,ANG MO KIO AVE 1,07 TO 09,82.0,New Generation,1977,57,221935
2,2018,ANG MO KIO,3 ROOM,220,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1977,57,221936
3,2018,ANG MO KIO,3 ROOM,445,ANG MO KIO AVE 10,01 TO 03,67.0,New Generation,1979,60,221937
4,2018,ANG MO KIO,3 ROOM,471,ANG MO KIO AVE 10,01 TO 03,67.0,New Generation,1979,59,221938


In [170]:
test_attributesName_Numeric=['month','lease_commence_date','remaining_lease','floor_area_sqm']
ChangeDataType(test_df,train1_attributesName_Category,'category')
ChangeDataType(test_df,test_attributesName_Numeric,'int64')
test_df["block"]=test_df["block"].astype("category")
test_df["storey_range"]=test_df["storey_range"].astype("category")
test_df["uniqueID"]=test_df["uniqueID"].astype("category")

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm          float64
flat_model             category
lease_commence_date       int64
remaining_lease          object
uniqueID               category
dtype: object
month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
remaining_lease           int64
uniqueID               category
dtype: object


In [173]:
test_df.dtypes
test_df.drop('uniqueID',axis=1,inplace=True)

In [174]:
test_df.dtypes

month                     int64
town                   category
flat_type              category
block                  category
street_name            category
storey_range           category
floor_area_sqm            int64
flat_model             category
lease_commence_date       int64
remaining_lease           int64
dtype: object

#### Merge Train 1,2,3 data frame
##### The Uniqe ID values are different in each data set, so it cont be merged, so trying to append the data frame

In [172]:
print(train_df1.columns)
print(train_df2.columns)
print(train_df3.columns)
print(test_df.columns)

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'uniqueID'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'uniqueID'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'uniqueID'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'uniqueID'],
      dtype='object')


In [140]:
trainData_main_df=MergeData(train_df1,train_df2,"uniqueID")

In [141]:
trainData_main_df=MergeData(trainData_main_df,train_df3,"uniqueID")

In [142]:
trainData_main_df.head()

,month_x,town_x,flat_type_x,block_x,street_name_x,storey_range_x,floor_area_sqm_x,flat_model_x,lease_commence_date_x,resale_price_x,...,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,uniqueID


In [143]:
temp1 = train_df1
temp2 = train_df2
temp3 = train_df3
train_main_df=temp1.append(temp2, ignore_index=True)
train_main_df=train_main_df.append(temp3,ignore_index=True)

In [144]:
train_main_df

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town,uniqueID
0,172,Improved,2 ROOM,45,1986,2012,73,250000,06 TO 10,ANG MO KIO AVE 4,ANG MO KIO,100000
1,510,Improved,2 ROOM,44,1980,2012,67,265000,01 TO 05,ANG MO KIO AVE 8,ANG MO KIO,100001
2,610,New Generation,3 ROOM,68,1980,2012,67,315000,06 TO 10,ANG MO KIO AVE 4,ANG MO KIO,100002
3,474,New Generation,3 ROOM,67,1984,2012,71,320000,01 TO 05,ANG MO KIO AVE 10,ANG MO KIO,100003
4,604,New Generation,3 ROOM,67,1980,2012,67,321000,06 TO 10,ANG MO KIO AVE 5,ANG MO KIO,100004
5,154,New Generation,3 ROOM,68,1981,2012,68,321000,01 TO 05,ANG MO KIO AVE 5,ANG MO KIO,100005
6,110,New Generation,3 ROOM,67,1978,2012,65,323000,01 TO 05,ANG MO KIO AVE 4,ANG MO KIO,100006
7,445,New Generation,3 ROOM,67,1979,2012,66,325000,01 TO 05,ANG MO KIO AVE 10,ANG MO KIO,100007
8,476,New Generation,3 ROOM,67,1979,2012,66,328000,11 TO 15,ANG MO KIO AVE 10,ANG MO KIO,100008
9,631,New Generation,3 ROOM,67,1985,2012,72,330000,01 TO 05,ANG MO KIO AVE 4,ANG MO KIO,100009


In [145]:
GetValueCounts(train_main_df['storey_range'])

04 TO 06    27861
07 TO 09    25062
01 TO 03    22431
10 TO 12    21733
13 TO 15     9564
16 TO 18     3933
01 TO 05     2700
06 TO 10     2474
19 TO 21     1735
11 TO 15     1259
22 TO 24     1170
25 TO 27      615
28 TO 30      407
16 TO 20      265
34 TO 36      169
37 TO 39      157
31 TO 33      149
21 TO 25       92
40 TO 42       76
26 TO 30       39
46 TO 48       15
43 TO 45       12
36 TO 40        7
49 TO 51        7
31 TO 35        2
Name: storey_range, dtype: int64


In [146]:
# DrawBarChart(train_main_df,train_main_df.month,train_main_df.resale_price,"Year","Resale-Price","VS")

In [147]:
# GenerateHeatMap(train_main_df)

In [148]:
train_main_df["block"]=train_main_df["block"].astype("category")
train_main_df["flat_model"]=train_main_df["flat_model"].astype("category")
train_main_df["storey_range"]=train_main_df["storey_range"].astype("category")
train_main_df["street_name"]=train_main_df["street_name"].astype("category")
train_main_df["uniqueID"]=train_main_df["uniqueID"].astype("category")

In [149]:
train_main_df.dtypes
FindMissingData(train_main_df,True,"train_main_df")

,percent_missing
column_name,
block,0.0
flat_model,0.0
flat_type,0.0
floor_area_sqm,0.0
lease_commence_date,0.0
month,0.0
remaining_lease,0.0
resale_price,0.0
storey_range,0.0


None


In [150]:
#Dropping Unique ID.
# DropAttributes(train_main_df,train_main_df.uniqueID)
# train_main_df.drop(train_main_df.uniqueID,axis=1,inplace= True)
train_main_df.drop('uniqueID',axis=1,inplace=True)
test_df.drop('uniqueID',axis=1,inplace=True)

In [151]:
cat_Attribute=list(train_main_df.select_dtypes("category").columns)
num_Attributes = list(train_main_df.columns.difference(cat_Attribute))
num_Attributes.pop()
print(cat_Attribute)
print(num_Attributes)

['block', 'flat_model', 'flat_type', 'storey_range', 'street_name', 'town']
['floor_area_sqm', 'lease_commence_date', 'month', 'remaining_lease']


In [185]:
X = train_main_df.drop('resale_price', axis=1)
Y = train_main_df['resale_price']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20,random_state=123)  

### Pipeline

In [153]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_Attributes),
        ('cat', categorical_transformer, cat_Attribute)])

### XG Boost

In [154]:
xg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),('xgboost',XGBRegressor())])

In [155]:
Xgbm_param_grid = {}
#'xgboost__max_depth': [8,10,12,14], 'xgboost__subsample': [0.8, 0.6,], 'xgboost__max_features':[0.2, 0.3], 
#               'xgboost__n_estimators': [10, 20, 30]
Xgbm_grid = GridSearchCV(xg_pipeline, param_grid=Xgbm_param_grid, cv=3)

In [156]:
Xgbm_grid.fit(X_train,Y_train)

[15:20:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:20:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:20:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:21:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                    

In [157]:
train_Xgbm_pred = Xgbm_grid.predict(X_test)
print(Xgbm_grid.score(X_test, Y_test))

0.8514128575098778


In [160]:
rms = sqrt(mean_squared_error(Y_test, train_Xgbm_pred))

In [161]:
rms

52243.47766303577

In [175]:
train_Xgbm_pred_actual = Xgbm_grid.predict(test_df)
train_Xgbm_pred_actual

array([239579.55, 374184.38, 314262.25, ..., 595439.7 , 621174.  ,
       608878.3 ], dtype=float32)

In [178]:
ExportFinal_CSV(test_df1['uniqueID'],train_Xgbm_pred_actual,"UniqueID","Resale_price")

,UniqueID,Resale_price
0,221934,239579.546875
1,221935,374184.375000
2,221936,314262.250000
3,221937,293416.562500
4,221938,293416.562500
5,221939,314262.250000
6,221940,293416.562500
7,221941,374184.375000
8,221942,306240.187500
9,221943,306240.187500


#### Random Forest

In [182]:
randomForestPipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', RandomForestRegressor())])

In [187]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=143)

param_grid = {}

randomForest_grid = GridSearchCV(randomForestPipeline, param_grid= param_grid, cv=kfold)

In [188]:
randomForest_grid.fit(X_train,Y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=143, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                

In [195]:
test_pred = randomForest_grid.predict(X_test)
print(randomForest_grid.score(X_test, Y_test))

0.9629512583731248


In [196]:
test_pred_actual2 = randomForest_grid.predict(test_df)

In [198]:
test_pred_actual2

array([222528.57142857, 343900.        , 281700.        , ...,
       622488.8       , 733666.4       , 714988.8       ])

In [199]:
rms = sqrt(mean_squared_error(Y_test,test_pred))
rms

26087.243628950724

In [201]:
ExportFinal_CSV(test_df1['uniqueID'],test_pred_actual2,"UniqueID","Resale_price","Mith_prediction2.csv")

,UniqueID,Resale_price
0,221934,222528.571429
1,221935,343900.000000
2,221936,281700.000000
3,221937,251100.000000
4,221938,253900.000000
5,221939,279233.333333
6,221940,258100.000000
7,221941,350400.000000
8,221942,281700.000000
9,221943,271200.000000
